In [2]:
# Importando los módulos que necesitaremos
import numpy as np

import pandas as pd
from pandas.plotting import table

import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

import seaborn as sns

from datetime import datetime

import os

# Configurando los estílos de los gráficos
plt.ioff()
sns.set_context('talk')
sns.set_style("whitegrid")

# Definiendo Constantes
BINS = [0, 10, 20, 30, 40, 50, 60, 70, 80, 300]

## Limpieza de Datos

In [3]:
# Leo la fuente de datos local en CSV 
df = pd.read_csv('data/vacunas_covid.csv')
df.head()

,FECHA_CORTE,UUID,GRUPO_RIESGO,EDAD,SEXO,FECHA_VACUNACION,DOSIS,FABRICANTE,DIRESA,DEPARTAMENTO,PROVINCIA,DISTRITO
0,20210309,c7ba6f9424c59a543e8803495c1061db,PERSONAL DE SALUD,39.0,MASCULINO,20210225,1,SINOPHARM,LAMBAYEQUE,LAMBAYEQUE,CHICLAYO,CHICLAYO
1,20210309,51ffbe83478f1e64da17fe2756a9594b,PERSONAL DE SALUD,37.0,FEMENINO,20210219,1,SINOPHARM,LAMBAYEQUE,LAMBAYEQUE,CHICLAYO,JOSE LEONARDO ORTIZ
2,20210309,54048fe3b06c35f14350d7e81340bdbd,PERSONAL DE SALUD,44.0,MASCULINO,20210220,1,SINOPHARM,LAMBAYEQUE,LAMBAYEQUE,CHICLAYO,JOSE LEONARDO ORTIZ
3,20210309,746ad64c513b5d53b987ed48a6b45f77,PERSONAL DE SALUD,54.0,FEMENINO,20210220,1,SINOPHARM,LAMBAYEQUE,LAMBAYEQUE,CHICLAYO,JOSE LEONARDO ORTIZ
4,20210309,80f0a41dd63af3b96d4a80bfab2eaa61,PERSONAL DE SALUD,35.0,FEMENINO,20210225,1,SINOPHARM,LAMBAYEQUE,LAMBAYEQUE,CHICLAYO,CHICLAYO


In [8]:
# Obtengo el conteo del número de valores en cada columnas
total_count = len(df)
print(f'TOTAL COUNT: {total_count}')
print(df.count())

Total: 423934
FECHA_CORTE         423934
UUID                423934
GRUPO_RIESGO        423934
EDAD                423454
SEXO                423934
FECHA_VACUNACION    423934
DOSIS               423934
FABRICANTE          423934
DIRESA              423934
DEPARTAMENTO        423934
PROVINCIA           423934
DISTRITO            423934
dtype: int64


In [4]:
# Verificamos que las columnas DEPARTAMENTO, GRUPO_RIESGO y SEXO tengan los valores indicados
print(df['DEPARTAMENTO'].unique())
print(df['GRUPO_RIESGO'].unique())
print(df['SEXO'].unique())

['LAMBAYEQUE' 'LIMA' 'AMAZONAS' 'LA LIBERTAD' 'CAJAMARCA' 'UCAYALI'
 'AYACUCHO' 'APURIMAC' 'SAN MARTIN' 'PIURA' 'HUANUCO' 'MADRE DE DIOS'
 'LORETO' 'CALLAO' 'CUSCO' 'ICA' 'HUANCAVELICA' 'AREQUIPA' 'JUNIN' 'PUNO'
 'PASCO' 'ANCASH' 'MOQUEGUA' 'TACNA' 'TUMBES']
['PERSONAL DE SALUD' 'TRABAJADOR Ó PERSONAL DE LIMPIEZA'
 'PERSONAL DE SEGURIDAD' 'PERSONAL MILITAR Ó FF AA'
 'POLICIA NACIONAL DEL PERU' 'ESTUDIANTES DE CIENCIAS DE LA SALUD'
 'BRIGADISTAS']
['MASCULINO' 'FEMENINO']


In [5]:
# Filtramos las filas que no tienen valor en las columnas EDAD o SEXO
df = df[df['EDAD'].notnull() & df['SEXO'].notnull()]

## Transformación de Datos

In [6]:
# Abreviando los grupos de riesgo
df['GRUPO_RIESGO'].replace('PERSONAL DE SALUD','P. SALUD', inplace=True)
df['GRUPO_RIESGO'].replace('TRABAJADOR Ó PERSONAL DE LIMPIEZA','T. LIMP.', inplace=True)
df['GRUPO_RIESGO'].replace('PERSONAL DE SEGURIDAD','P. SEG.', inplace=True)
df['GRUPO_RIESGO'].replace('PERSONAL MILITAR Ó FF AA','FF.AA.', inplace=True)
df['GRUPO_RIESGO'].replace('POLICIA NACIONAL DEL PERU','P.N.P.', inplace=True)
df['GRUPO_RIESGO'].replace('ESTUDIANTES DE CIENCIAS DE LA SALUD','E. SALUD', inplace=True)
df['GRUPO_RIESGO'].replace('BRIGADISTAS','BRIG.', inplace=True)

# Genero Serie de grupos de riesgo
grupos_riesgo = df['GRUPO_RIESGO'].unique()
grupos_riesgo_series = pd.Series(np.zeros_like(grupos_riesgo),grupos_riesgo)

# Agrupo el conjunto de datos por DEPARTAMENTO, SEXO y EDAD para el DataFrame
gb_age = df[["DEPARTAMENTO","SEXO",'EDAD']].groupby(['DEPARTAMENTO', 'SEXO', pd.cut(df['EDAD'], BINS, False)])

# Agrupo el conjunto de datos por DEPARTAMENTO, SEXO y GRUPO_RIESGO para el DataFrame
gb_gr = df[["DEPARTAMENTO","SEXO",'GRUPO_RIESGO']].groupby(['DEPARTAMENTO', 'SEXO', 'GRUPO_RIESGO'])

In [7]:
def get_data_to_plot(department=None):
    if(department is None):
        return df[['SEXO','EDAD','GRUPO_RIESGO','FECHA_VACUNACION']]
    else:
        return df[df['DEPARTAMENTO'] == department][['SEXO','EDAD','GRUPO_RIESGO','FECHA_VACUNACION']]

## Visualización de Datos

In [8]:
# Visualizo los datos agrupados por EDAD via DataFrame
gb_age.size().unstack()

EDAD                     [0, 10)  [10, 20)  [20, 30)  [30, 40)  [40, 50)  \
DEPARTAMENTO  SEXO                                                         
AMAZONAS      FEMENINO         0        10       771      1430       832   
              MASCULINO        0         8       394       750       547   
ANCASH        FEMENINO         0         1      1498      2680      1969   
              MASCULINO        0         9       582      1262       967   
APURIMAC      FEMENINO         0         3      1036      1967      1589   
              MASCULINO        0         5       390       950       989   
AREQUIPA      FEMENINO         0         6      2165      4212      3048   
              MASCULINO        0         4       724      1662      1331   
AYACUCHO      FEMENINO         0         7      1134      2287      1728   
              MASCULINO        0         6       420      1077      1091   
CAJAMARCA     FEMENINO         0         2      1978      3274      2152   
              MASCULINO        0        12       858      1772      1243   
CALLAO        FEMENINO         0        20      1675      4282      3732   
              MASCULINO        0        34      1129      2129      2148   
CUSCO         FEMENINO         0        10      1292      3492      2335   
              MASCULINO        0         9       532      1403      1297   
HUANCAVELICA  FEMENINO         0         6      1035      1632       953   
              MASCULINO        0         4       392       756       607   
HUANUCO       FEMENINO         0         6      1299      2221      1466   
              MASCULINO        0        15       514      1241       906   
ICA           FEMENINO         0         5      1508      2291      1543   
              MASCULINO        0         6       516      1028       791   
JUNIN         FEMENINO         0         5      2810      4301      2594   
              MASCULINO        0        15       884      1857      1365   
LA LIBERTAD   FEMENINO         0         6      2583      5062      3427   
              MASCULINO        0        17      1179      2309      1909   
LAMBAYEQUE    FEMENINO         0         2      1430      2904      2239   
              MASCULINO        0         5       576      1312      1265   
LIMA          FEMENINO         0       196     13351     31449     25180   
              MASCULINO        0       639      8468     15714     15534   
LORETO        FEMENINO         0         5       757      1813      1242   
              MASCULINO        0        12       592      1233       827   
MADRE DE DIOS FEMENINO         0         4       220       488       358   
              MASCULINO        0         1        87       243       177   
MOQUEGUA      FEMENINO         0         4       564       992       792   
              MASCULINO        0         5       187       413       334   
PASCO         FEMENINO         0         2       575       843       619   
              MASCULINO        0         5       205       394       377   
PIURA         FEMENINO         0        13      1965      3715      3200   
              MASCULINO        0        16       913      1785      1663   
PUNO          FEMENINO         0         5      1168      2417      1641   
              MASCULINO        0         1       401      1045       935   
SAN MARTIN    FEMENINO         0        33      1502      2809      1408   
              MASCULINO        0        24       829      1672       961   
TACNA         FEMENINO         0         4       582      1092      1033   
              MASCULINO        0         8       296       506       456   
TUMBES        FEMENINO         0         0       411       674       566   
              MASCULINO        0         1       204       356       297   
UCAYALI       FEMENINO         0         6       745      1191       856   
              MASCULINO        0         9       462       726       577   

EDAD                     [50, 60)  [60, 70)  [7

In [9]:
# Visualizo los datos agrupados por GRUPO_RIESGO via DataFrame
gb_gr.size().unstack()

GRUPO_RIESGO             BRIG.  E. SALUD  FF.AA.  P. SALUD  P. SEG.  P.N.P.  \
DEPARTAMENTO  SEXO                                                            
AMAZONAS      FEMENINO     NaN       NaN    10.0    3511.0      8.0    18.0   
              MASCULINO    NaN       NaN    18.0    2041.0     34.0    11.0   
ANCASH        FEMENINO     NaN      39.0     NaN    8126.0      5.0     4.0   
              MASCULINO    NaN      26.0     5.0    3830.0     73.0     1.0   
APURIMAC      FEMENINO     NaN       NaN     NaN    5528.0     24.0     2.0   
              MASCULINO    NaN       NaN     NaN    3062.0     22.0     NaN   
AREQUIPA      FEMENINO     NaN       NaN     NaN   13599.0      5.0     1.0   
              MASCULINO    NaN       NaN     NaN    5586.0     55.0     2.0   
AYACUCHO      FEMENINO     NaN       4.0     NaN    6416.0     16.0     5.0   
              MASCULINO    NaN       3.0     NaN    3610.0     56.0     2.0   
CAJAMARCA     FEMENINO     NaN      15.0     NaN    8639.0     67.0     1.0   
              MASCULINO    NaN      12.0     NaN    4738.0    121.0     1.0   
CALLAO        FEMENINO     NaN       NaN    48.0   13768.0     20.0   149.0   
              MASCULINO    NaN       NaN    19.0    7260.0     96.0   883.0   
CUSCO         FEMENINO     NaN       NaN     NaN    9368.0     11.0     5.0   
              MASCULINO    NaN       1.0     4.0    4481.0     48.0     5.0   
HUANCAVELICA  FEMENINO     NaN       NaN     NaN    4178.0      1.0     1.0   
              MASCULINO    NaN       NaN     4.0    2185.0      9.0     NaN   
HUANUCO       FEMENINO     NaN       4.0     NaN    5973.0     55.0     3.0   
              MASCULINO    NaN       3.0     NaN    3331.0     90.0     1.0   
ICA           FEMENINO     NaN      53.0     1.0    7145.0     29.0    42.0   
              MASCULINO    NaN      22.0     NaN    3332.0     99.0    25.0   
JUNIN         FEMENINO     NaN     137.0    12.0   11932.0     14.0    85.0   
              MASCULINO    NaN      40.0    23.0    5388.0     94.0    76.0   
LA LIBERTAD   FEMENINO     NaN      15.0     NaN   14564.0     17.0     3.0   
              MASCULINO    NaN       8.0     NaN    7181.0    166.0     4.0   
LAMBAYEQUE    FEMENINO     NaN       NaN    16.0    8944.0      NaN     NaN   
              MASCULINO    NaN       NaN     4.0    4590.0      1.0     2.0   
LIMA          FEMENINO     1.0      22.0   959.0   96671.0    270.0  1195.0   
              MASCULINO    1.0      18.0  3507.0   48934.0    997.0  8594.0   
LORETO        FEMENINO     NaN       3.0     1.0    4771.0      NaN     5.0   
              MASCULINO    NaN       1.0     5.0    3404.0     21.0     5.0   
MADRE DE DIOS FEMENINO     NaN       NaN     4.0    1342.0     10.0     NaN   
              MASCULINO    NaN       NaN     8.0     716.0      5.0     NaN   
MOQUEGUA      FEMENINO     NaN       NaN     NaN    2958.0     38.0     1.0   
              MASCULINO    NaN       NaN     NaN    1344.0     66.0     NaN   
PASCO         FEMENINO     NaN       2.0     2.0    2479.0      7.0     NaN   
              MASCULINO    NaN       NaN     2.0    1250.0     68.0     NaN   
PIURA         FEMENINO     NaN       9.0    31.0   11197.0     25.0    17.0   
              MASCULINO    1.0      16.0    30.0    6071.0     36.0     9.0   
PUNO          FEMENINO     NaN       1.0     NaN    6958.0     78.0     NaN   
              MASCULINO    NaN       NaN     NaN    3651.0     39.0     NaN   
SAN MARTIN    FEMENINO     NaN       NaN     NaN    6935.0     40.0     NaN   
              MASCULINO    NaN       NaN     2.0    4445.0     67.0     1.0   
TACNA         FEMENINO     NaN       9.0     NaN    3864.0      2.0     2.0   
              MASCULINO    NaN       4.0     NaN    1943.0      3.0     1.0   
TUMBES        FEMENINO     NaN       NaN     NaN    1947.0      6.0     NaN   
              MASCULINO    NaN       NaN     NaN    1063.0     71.0     NaN   
UCAYALI       FEMENINO     NaN       NaN     NaN  

In [10]:
yticks = [5,15,25,35,45,55,65,75,85]

def generate_plot(area, data, date_range, show_plot=False, save_plot=True):

    # Creamos la figura, los ejes y agregamos la atribución
    plt.clf()

    fig, axs = plt.subplots(2,2, figsize=(20,20))
    plt.figtext(0.1,0.95, area + ': DISTRIBUCIÓN DE VACUNADOS CONTRA COVID-19 (N=' + str(len(data)) + ')', ha='left', fontsize=24)
    plt.figtext(0.1,0.925, 'Rango de fechas de vacunación: del {} al {}'.format(f'{date_range[0]:%Y-%m-%d}',f'{date_range[1]:%Y-%m-%d}'), ha='left', fontsize=20, color='#999')
    plt.figtext(0.9,0.02, """Fuente: https://www.datosabiertos.gob.pe/dataset/vacunaci%C3%B3n-contra-covid-19-ministerio-de-salud-minsa
    https://malexandersalazar.github.io/, Moisés Alexander Salazar Vila, """ + f'{datetime.now():%Y-%m-%d}', ha='right')

    # 1. Graficamos distribución por GRUPO ETAREO
    xlim_max = 0

    ## 1.1. Graficamos el primer histograma
    N, bins, patches = axs[0][0].hist(data["EDAD"], BINS, orientation = 'horizontal')

    ### Colorearemos las barras en base el conteo relativo de cada agrupación
    xlim_max = N.max() * 1.1
    fracs = N / N.max()
    norm = colors.Normalize(fracs.min(), fracs.max())

    ### Recorremos las barras y actualizamos cada color respectivamente
    for thisfrac, thispatch in zip(fracs, patches):
        color = plt.cm.viridis(norm(thisfrac))
        thispatch.set_facecolor(color)

    ### Personalizamos las ejes de nuestro gráfico
    axs[0][0].set_yticklabels([])
    axs[0][0].yaxis.set_tick_params(width=0)
    axs[0][0].set_yticks(yticks, minor=True)
    axs[0][0].set_yticklabels(['0-9','10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80+'], minor=True)
    axs[0][0].set_ylim(0,90)
    axs[0][0].set_xlim(0,xlim_max)
    axs[0][0].set_ylabel('GRUPO ETÁREO (AÑOS)')

    ### Calculamos y anotamos los porcentajes correspondientes a cada barra
    for i in range(len(BINS)-1):
        axs[0][0].text(N[i], yticks[i]-1, str(round(N[i]*100/len(data),2)) + '%')

    ## 1.2. Graficamos el segundo histograma
    males = data[data['SEXO'] == 'MASCULINO']["EDAD"]
    females = data[data['SEXO'] == 'FEMENINO']["EDAD"]

    N, bins, patches = axs[0][1].hist([males, females], BINS, orientation = 'horizontal')
    
    ### Personalizamos las ejes de nuestro gráfico
    axs[0][1].set_yticklabels([])
    axs[0][1].yaxis.set_tick_params(width=0)
    axs[0][1].set_yticks(yticks, minor=True)
    axs[0][1].set_ylim(0,90)
    axs[0][1].set_xlim(0,xlim_max)

    ### Calculamos y anotamos los porcentajes correspondientes a cada barra
    for i in range(len(BINS)-1):
        axs[0][1].text(N[0][i], yticks[i]-3, str(round(N[0][i]*100/len(data),2)) + '%')
        axs[0][1].text(N[1][i], yticks[i]+1, str(round(N[1][i]*100/len(data),2)) + '%')

    axs[0][1].legend(['Hombres (n=' + str(len(males)) + ')', 'Mujeres (n=' + str(len(females)) + ')'])

    # 2. Graficamos distribución por GRUPO RIESGO
    xlim_max = 0

    ## 2.1. Graficamos el primer gráfico de barras 
    group_counts = data['GRUPO_RIESGO'].value_counts().add(grupos_riesgo_series,fill_value=0)
    group_counts.sort_index(inplace=True,ascending=False)
    pos = np.array(range(len(group_counts)))
    patches = axs[1][0].barh(pos, group_counts.values, align='center')
    
    ### Colorearemos las barras en base el conteo relativo de cada agrupación
    xlim_max = group_counts.values.max() * 1.1
    fracs = group_counts.values / group_counts.values.max()
    norm = colors.Normalize(fracs.min(), fracs.max())

    ### Recorremos las barras y actualizamos cada color respectivamente
    for thisfrac, thispatch in zip(fracs, patches):
        color = plt.cm.viridis(norm(thisfrac))
        thispatch.set_facecolor(color)

    ### Personalizamos las ejes de nuestro gráfico
    axs[1][0].set_yticks(pos)
    axs[1][0].set_yticklabels(list(group_counts.index.values))
    axs[1][0].set_xlim(0,xlim_max)

    ### Calculamos y anotamos los porcentajes correspondientes a cada barra
    for i in range(len(pos)):
        axs[1][0].text(group_counts.values[i], pos[i]-0.1, str(round(group_counts[i]*100/len(data),2)) + '%')

    ## 2.2. Graficamos el segundo gráfico de barras 
    males = data[data['SEXO'] == 'MASCULINO']
    males_group_counts = males["GRUPO_RIESGO"].value_counts().add(grupos_riesgo_series,fill_value=0)
    males_group_counts.sort_index(inplace=True,ascending=False)

    females = data[data['SEXO'] == 'FEMENINO']
    females_group_counts = females["GRUPO_RIESGO"].value_counts().add(grupos_riesgo_series,fill_value=0)
    females_group_counts.sort_index(inplace=True,ascending=False)

    pos = np.array(range(len(males_group_counts)))

    axs[1][1].barh(pos, males_group_counts.values, 0.35, align='center')
    axs[1][1].barh(pos+0.35, females_group_counts.values, 0.35, align='center')
    
    ### Personalizamos las ejes de nuestro gráfico
    axs[1][1].set_yticks(pos)
    axs[1][1].set_yticklabels([])
    axs[1][1].set_xlim(0,xlim_max)
    
    ### Calculamos y anotamos los porcentajes correspondientes a cada barra
    for i in range(len(pos)):
        axs[1][1].text(males_group_counts.values[i], pos[i]-0.1, str(round(males_group_counts[i]*100/len(data),2)) + '%')
        axs[1][1].text(females_group_counts.values[i], pos[i]+0.3, str(round(females_group_counts[i]*100/len(data),2)) + '%')

    ## 3. Mostramos, guardamos y generamos el Markdown para las imágenes

    sns.despine(left=False, bottom=False)

    if save_plot:
        filename = 'dist/{}_{}.png'.format(f'{datetime.now():%Y%m%d}', area.replace(' ', '_'))
        plt.savefig("../" + filename, bbox_inches='tight')
        with open("../dist/images.txt", "a", encoding='utf-8') as f:
            f.write('![alt text]({} "{}")\n\n'.format(filename, area))

    if show_plot:
        plt.show()

In [11]:
# Eliminando archivo generado para el Markdown de las imágenes
if os.path.exists("../dist/images.txt"):
    os.remove("../dist/images.txt")

In [12]:
# Perú
data = get_data_to_plot()
dates = pd.to_datetime(data['FECHA_VACUNACION'], infer_datetime_format=False, format='%Y%m%d')
generate_plot('PERÚ', data, (dates.min(), dates.max()), False, True)

In [13]:
# Deparmanetos
departments_ordered = np.sort(df['DEPARTAMENTO'].unique())
for d in departments_ordered:
    data = get_data_to_plot(d)
    dates = pd.to_datetime(data['FECHA_VACUNACION'], infer_datetime_format=False, format='%Y%m%d')
    generate_plot(d, data, (dates.min(), dates.max()), False, True)

<ipython-input-10-bc64596a9b24>:8: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, axs = plt.subplots(2,2, figsize=(20,20))
